In [2]:
import os
import langextract as lx
from langextract_litellm import LiteLLMLanguageModel
import textwrap

In [3]:
prompt = textwrap.dedent("""\
    Extract characters, emotions, and relationships in order of appearance.
    Use exact text for extractions. Do not paraphrase or overlap entities.
    Provide meaningful attributes for each entity to add context.""")

# 2. Provide a high-quality example to guide the model
examples = [
    lx.data.ExampleData(
        text="ROMEO. But soft! What light through yonder window breaks? It is the east, and Juliet is the sun.",
        extractions=[
            lx.data.Extraction(
                extraction_class="character",
                extraction_text="ROMEO",
                attributes={"emotional_state": "wonder"}
            ),
            lx.data.Extraction(
                extraction_class="emotion",
                extraction_text="But soft!",
                attributes={"feeling": "gentle awe"}
            ),
            lx.data.Extraction(
                extraction_class="relationship",
                extraction_text="Juliet is the sun",
                attributes={"type": "metaphor"}
            ),
        ]
    )
]
input_text = "Lady Juliet gazed longingly at the stars, her heart aching for Romeo"

In [4]:
config = lx.factory.ModelConfig(
    model_id=f"litellm/azure/gpt-4o",
    provider="LiteLLMLanguageModel",
)
model = lx.factory.create_model(config)
results = list(model.infer([prompt]))
results

InferenceConfigError: Failed to create provider LiteLLMLanguageModel: Can't instantiate abstract class LiteLLMLanguageModel with abstract method infer